<a href="https://colab.research.google.com/github/AI4ALL-ESG-Investing/esg-financial-assistant/blob/main/data-cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning Notebook

# Import python libraries

In [2]:
import os
import pandas as pd
import yfinance as yf
import logging

logging.getLogger("yfinance").setLevel(logging.CRITICAL)


## Load Data into Colab

In [3]:
# Clone the GitHub repository
!git clone https://github.com/AI4ALL-ESG-Investing/esg-financial-assistant

# Change working directory to where the dataset is located
%cd esg-financial-assistant/data

os.listdir()

fatal: destination path 'esg-financial-assistant' already exists and is not an empty directory.
/Users/amandadongsmacbookpro/esg-financial-assistant/notebooks/esg-financial-assistant/data


['public-company-esg-ratings-dataset.csv',
 'README.md',
 'investment_survey.csv']

In [4]:
# clean ESG dataset
df = pd.read_csv("public-company-esg-ratings-dataset.csv")
df.head()

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,...,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,...,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,...,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,...,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,...,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258


In [5]:
df.drop(columns = ["logo", "weburl", "environment_grade", "environment_level", "social_grade", "social_level", "governance_grade", "governance_level", "total_grade", "total_level", "cik", "last_processing_date"], inplace = True)

In [6]:
df.isna().any()

ticker               False
name                 False
currency             False
exchange             False
industry              True
environment_score    False
social_score         False
governance_score     False
total_score          False
dtype: bool

In [7]:
row_na = df[df["industry"].isna()]

In [8]:
print(row_na["name"])

15                Armada Acquisition Corp I
27            Acri Capital Acquisition Corp
32         ACE Convergence Acquisition Corp
57                    Edoc Acquisition Corp
76                      AF Acquisition Corp
97                     AIB Acquisition Corp
101        Sports Ventures Acquisition Corp
123                Alignment Healthcare LLC
630       Health Assurance Acquisition Corp
646    Healthcare Services Acquisition Corp
669                Artisan Acquisition Corp
675                          Powered Brands
696                Concord Acquisition Corp
Name: name, dtype: object


In [9]:
industry_map = {
    'Armada Acquisition Corp I': 'Financial Services',
    'Acri Capital Acquisition Corp': 'Financial Services',
    'ACE Convergence Acquisition Corp': 'Technology',
    'Edoc Acquisition Corp': 'Healthcare',
    'AF Acquisition Corp': 'Financial Services',
    'AIB Acquisition Corp': 'Financial Services',
    'Sports Ventures Acquisition Corp': 'Media & Entertainment',
    'Alignment Healthcare LLC': 'Healthcare',
    'Health Assurance Acquisition Corp': 'Healthcare',
    'Healthcare Services Acquisition Corp': 'Healthcare',
    'Artisan Acquisition Corp': 'Financial Services',
    'Powered Brands': 'Consumer Goods',
    'Concord Acquisition Corp': 'Financial Services'
}
df['industry'] = df.apply(
    lambda row: industry_map[row['name']] if pd.isna(row['industry']) and row['name'] in industry_map else row['industry'],
    axis=1
)


In [10]:
df.isna().any()
# ESG Dataset cleaned

ticker               False
name                 False
currency             False
exchange             False
industry             False
environment_score    False
social_score         False
governance_score     False
total_score          False
dtype: bool

In [11]:
import yfinance as yf
import pandas as pd

tickers = df["ticker"].unique().tolist()

# 批量下载最新价格（1天）
hist = yf.download(tickers, period="1d", group_by='ticker', progress=False)

latest_prices = {}
for ticker in tickers:
    try:
        # 这里不同ticker格式可能不同，单ticker会返回DataFrame，多ticker是多层索引
        if len(tickers) == 1:
            close_price = hist["Close"].iloc[-1]
        else:
            close_price = hist[ticker]["Close"].iloc[-1]
        latest_prices[ticker] = close_price
    except Exception:
        latest_prices[ticker] = None

# 请求beta仍要单独请求info，建议用简单的串行或并行
betas = {}
for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        betas[ticker] = info.get("beta", None)
    except Exception:
        betas[ticker] = None

# 转DataFrame
price_beta_df = pd.DataFrame({
    "ticker": tickers,
    "Latest_Price": [latest_prices.get(t, None) for t in tickers],
    "Beta": [betas.get(t, None) for t in tickers]
})

stock_merged = pd.merge(df, price_beta_df, on="ticker", how="inner")
print(stock_merged.head())


/var/folders/0m/frmvql3x50bckqpz6zv0t7800000gp/T/ipykernel_12902/2848369013.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  hist = yf.download(tickers, period="1d", group_by='ticker', progress=False)


  ticker                           name currency  \
0    dis                 Walt Disney Co      USD   
1     gm              General Motors Co      USD   
2    gww                WW Grainger Inc      USD   
3    mhk          Mohawk Industries Inc      USD   
4    lyv  Live Nation Entertainment Inc      USD   

                        exchange                            industry  \
0  NEW YORK STOCK EXCHANGE, INC.                               Media   
1  NEW YORK STOCK EXCHANGE, INC.                         Automobiles   
2  NEW YORK STOCK EXCHANGE, INC.  Trading Companies and Distributors   
3  NEW YORK STOCK EXCHANGE, INC.                   Consumer products   
4  NEW YORK STOCK EXCHANGE, INC.                               Media   

   environment_score  social_score  governance_score  total_score  \
0                510           316               321         1147   
1                510           303               255         1068   
2                255           385             

In [12]:
stock_merged.dropna(subset=['Latest_Price'], inplace = True)
print(stock_merged.head(20))

Empty DataFrame
Columns: [ticker, name, currency, exchange, industry, environment_score, social_score, governance_score, total_score, Latest_Price, Beta]
Index: []


In [13]:
# stock_merged.isna().any()